In [ ]:
%cd /home/ltchen/gnnpp
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import pytorch_lightning as L
import torch_geometric
import wandb

from torch.utils.data import DataLoader
from torch.optim import AdamW
from models.graphensemble.multigraph import Multigraph
from utils.data import (
    load_dataframes,
    load_distances,
    normalize_features_and_create_graphs,
    rm_edges,
    summary_statistics,
)

In [ ]:
DIRECTORY = os.getcwd()
SAVEPATH = os.path.join(DIRECTORY, "explored_models/gnn_24h/models")

In [ ]:
# build a graph => create multigraph - without summmary_statistics and no edges removed
max_dist =

dataframes = load_dataframes(mode="train", leadtime="24h")
dist = load_distances(dataframes["stations"])

graphs_train_rf, tests = normalize_features_and_create_graphs(
    training_data=dataframes["train"],
    valid_test_data=[dataframes["test_rf"], dataframes["test_f"]],
    mat=dist,
    max_dist=max_dist,
)
graphs_test_rf, graphs_test_f = tests

# wo ist alles was hinter diesem Code kommt : pop stations, reset_index, dropnans in normalize_features_and_create_graphs
dataframes.pop("stations") # .pop("stations") => entfernt den df mit stations, wofuer brauche ich die dann überhaupt? Grafik?

# test
for X, y in dataframes.values(): # wofuer?
    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)

train, valid_test = normalize_features_and_create_graphs(
    training_data=dataframes["train"], valid_test_data=[dataframes["test_rf"], dataframes["test_f"]]
)

train = drop_nans(train)
(test_rf, test_f) = valid_test
test_rf = drop_nans(test_rf)
test_f = drop_nans(test_f)

DIRECTORY = os.getcwd()
SAVEPATH = os.path.join(DIRECTORY, "explored_models/drn_24h/models")

In [ ]:
max_epochs =
batch_size = 512
emb_dim = 20
gnn_hidden =
gnn_layers =
heads =
lr =

print("[INFO] Creating data loaders...")
train_loader = DataLoader(graphs_train_rf, batch_size=batch_size, shuffle=True)

print("[INFO] Creating model...")

in_channels = graphs_train_rf[0].x.shape[1] + emb_dim - 1

multigraph = Multigraph(
    embedding_dim=emb_dim,
    in_channels=in_channels,
    hidden_channels_gnn=gnn_hidden,
    out_channels_gnn=gnn_hidden,
    num_layers_gnn=gnn_layers,
    heads=heads,
    hidden_channels_deepset=gnn_hidden,
    optimizer_class=AdamW,
    optimizer_params=dict(lr=lr),
)
torch_geometric.compile(multigraph)

# understand what this is
batch = next(iter(train_loader))
batch = batch  # .to("cuda")
multigraph  # .to("cuda")
multigraph.forward(batch)

wandb_logger = WandbLogger(project="multigraph")
os_checkpoint_callback = ModelCheckpoint(
    dirpath=SAVEPATH, filename=f"run_24h", monitor="train_loss", mode="min", save_top_k=1
)

print("[INFO] Training model...")
trainer = L.Trainer(
    max_epochs=max_epochs,
    log_every_n_steps=1,
    accelerator="gpu",
    devices=1,
    enable_progress_bar=True,
    logger=wandb_logger,
    callbacks=checkpoint_callback,
)

trainer.fit(model=multigraph, train_dataloaders=train_loader)

In [ ]:
# gnn architecture

In [ ]:
# train gnn